#Scraping data from reddit

Reddit has data on multiple topics that can be used for analysis and understanding of specific topic or issue. People post about their interest and experience.

Access:

Under App Preferences, create a new app with type script. This will give access to a temporary OAuth token (client ID and secret ID) . Apart from that, we will also need the username and password for the reddit app to access the API. 

In [1]:
import requests
import pandas as pd

In [2]:
with open('auth.txt') as f:
  lines= f.readlines()

CLIENT_ID= lines[0].replace('\n','')
SECRET_KEY = lines[1].replace('\n','')
username = lines[2].replace('\n','')
password = lines[3].replace('\n','')

In [3]:
auth= requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

data ={
    'grant_type':'password',
    'username':username,
    'password' : password
}

headers = {'User-Agent':'ScraperAPI'}

In [4]:
def get_access_token():
  client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
  post_data = data
  # send request for an OAuth token
  response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
  # Convert response to json and return the access token
  return response.json()['access_token']

In [5]:
TOKEN= get_access_token()


In [6]:
# add authorization to our headers dictionary
# headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
headers['Authorization'] =f'bearer {TOKEN}'

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [7]:
def append_results(res,df):
  for post in res.json()['data']['children']:
    df= df.append({
        'id':post['kind'] + '_' + post['data']['id'],
        'subreddit':post['data']['subreddit'],
        'title': post['data']['title'],
        'text' : post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups' : post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score']
    },ignore_index=True)

  return df

In [8]:
res = requests.get('https://oauth.reddit.com/r/depression/new', headers=headers,
                   params={'limit':'1000'})

In [9]:
df = pd.DataFrame()
df= append_results(res,df)
len(df)

100

In [ ]:
for i in range(50):
  last_post_id = res.json()['data']['children'][-1]
  last_post_id = last_post_id['kind'] + '_' + last_post_id['data']['id']

  print(last_post_id)

  res = requests.get('https://oauth.reddit.com/r/depression/new', headers=headers,
                    params={'after': last_post_id})

  df = append_results(res,df)

  print(len(df))

In [11]:
df.to_csv('depression.csv')